In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using Theano backend.


### Setup

In [2]:
text = open('../data/nietzsche.txt').read();

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1

In [4]:
chars.insert(0, '\0')

In [5]:
''.join(chars)[1:-6]

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join([indices_char[i] for i in idx[:50]])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what th'

### 3 char model

In [10]:
cs=3
c1_dat = [idx[i] for i in xrange(0, len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0, len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0, len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0, len(idx)-1-cs, cs)]

In [11]:
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [12]:
y = np.stack(c4_dat[:-2])

In [13]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [14]:
y[:4]

array([30, 29,  1, 40])

In [15]:
x1.shape, y.shape

((200297,), (200297,))

In [16]:
n_fac = 42

In [17]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [18]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

### Create and train model

In [19]:
n_hidden = 256

In [20]:
dense_in = Dense(n_hidden, activation='relu')

In [21]:
c1_hidden = dense_in(c1)

In [22]:
dense_hidden = Dense(n_hidden, activation='relu')

In [23]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

In [24]:
c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

In [25]:
dense_out = Dense(vocab_size, activation='softmax')

In [26]:
c4_out = dense_out(c3_hidden)

In [27]:
model = Model([c1_in, c2_in, c3_in], c4_out)

In [28]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [29]:
model.optimizer.lr = 0.00001

In [30]:
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4)

Epoch 1/4
200297/200297 [==============================] - 21s - loss: 3.5112    - ETA: 0s - loss: 3
Epoch 2/4
200297/200297 [==============================] - 21s - loss: 3.0270    
Epoch 3/4
200297/200297 [==============================] - 21s - loss: 2.9259    
Epoch 4/4
200297/200297 [==============================] - 20s - loss: 2.8235    


### Test model

In [31]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict(arrs)
    i = np.argmax(p)
    print(p)
    print(i)
    return chars[i]

In [32]:
get_next(' an')

[[  1.7250e-06   2.0914e-02   2.7957e-01   4.0563e-04   2.8950e-03   3.4384e-04   2.5632e-04
    2.8968e-04   1.4046e-02   5.4752e-03   5.0328e-03   6.6023e-05   1.8384e-04   1.3234e-04
    9.1259e-05   9.2757e-05   8.1668e-05   6.2487e-05   8.0585e-05   7.3310e-05   6.0326e-05
    8.3221e-04   6.7338e-04   2.9418e-04   4.9045e-04   9.3712e-04   3.1741e-04   4.0234e-04
    4.0012e-04   1.4128e-03   4.4263e-04   3.9969e-04   4.0089e-04   1.4348e-03   5.5398e-05
    7.3345e-05   3.4183e-04   4.1197e-04   6.7931e-04   9.2405e-04   4.2263e-04   4.4240e-05
    7.6321e-04   9.4573e-04   1.5756e-03   3.7149e-04   1.1130e-04   5.1885e-04   3.5028e-05
    9.4957e-05   1.3729e-05   8.0879e-05   1.3634e-04   1.6832e-04   3.6683e-02   5.6629e-03
    2.1633e-02   7.1235e-02   1.1526e-01   9.9678e-03   1.6607e-02   1.1769e-02   5.0402e-02
    6.6347e-04   3.3807e-03   3.2106e-02   1.5824e-02   2.3639e-02   3.3157e-02   1.2412e-02
    6.5995e-04   2.9130e-02   5.5050e-02   5.6534e-02   1.6671e-02   7

' '

**Holy shit, this does not work!**

### Our first RNN!

#### Create inputs

In [33]:
cs=8

In [34]:
c_in_dat = [ [idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] for n in range(cs) ]

In [35]:
print(idx[:40])

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1, 43, 45, 40, 40, 39, 43, 33, 38, 31, 2, 73, 61, 54, 73, 2, 44, 71, 74, 73, 61, 2, 62, 72, 2, 54, 2, 76, 68, 66, 54]


In [36]:
for i in range(len(c_in_dat)):
    print(c_in_dat[i][:20])

[40, 1, 33, 2, 72, 67, 73, 2, 68, 57, 72, 2, 65, 68, 62, 71, 78, 58, 54, 61]
[42, 1, 38, 44, 2, 9, 61, 73, 73, 1, 69, 73, 2, 69, 67, 2, 2, 58, 73, 54]
[29, 43, 31, 71, 54, 9, 58, 61, 2, 59, 58, 61, 69, 61, 2, 54, 61, 67, 62, 75]
[30, 45, 2, 74, 2, 76, 67, 58, 60, 68, 56, 54, 61, 58, 72, 72, 54, 1, 72, 58]
[25, 40, 73, 73, 76, 61, 24, 71, 71, 71, 73, 73, 62, 71, 68, 2, 75, 57, 73, 2]
[27, 40, 61, 61, 68, 54, 2, 58, 68, 2, 62, 2, 65, 72, 2, 73, 58, 68, 72, 59]
[29, 39, 54, 2, 66, 73, 33, 2, 74, 72, 67, 54, 68, 8, 59, 61, 2, 60, 8, 54]
[1, 43, 73, 62, 54, 2, 72, 67, 67, 74, 60, 65, 72, 2, 54, 58, 55, 66, 2, 62]


In [37]:
c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

In [38]:
xs = [np.stack(c[:-2]) for c in c_in_dat]

In [39]:
len(xs), xs[0].shape

(8, (75110,))

In [40]:
y = np.stack(c_out_dat[:-2])

In [41]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [42]:
y[:cs]

array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [43]:
n_fac = 42

### Create and train model

In [44]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [45]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

In [46]:
n_hidden = 256

In [47]:
dense_in = Dense(n_hidden, activation='relu', name="dense_in")
dense_hidden = Dense(n_hidden, activation='relu', init='identity', name="dense_hidden")
dense_out = Dense(vocab_size, activation='softmax', name="dense_out")

In [48]:
hidden = dense_in(c_ins[0][1])

In [49]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [50]:
c_out = dense_out(hidden)

In [51]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [52]:
from keras.utils import visualize_util
visualize_util.plot(model, to_file='model.png')
# plot_model(model, to_file='model.png')

In [53]:
model.fit(xs, y, batch_size=64, nb_epoch=5)

Epoch 1/5
75110/75110 [==============================] - 15s - loss: 2.5366    
Epoch 2/5
75110/75110 [==============================] - 15s - loss: 2.2553    
Epoch 3/5
75110/75110 [==============================] - 17s - loss: 2.1515    
Epoch 4/5
75110/75110 [==============================] - 19s - loss: 2.0803    
Epoch 5/5
75110/75110 [==============================] - 19s - loss: 2.0243    


In [54]:
model.fit(xs, y, batch_size=64, nb_epoch=5)

Epoch 1/5
75110/75110 [==============================] - 20s - loss: 1.9767    - ETA: 0s - loss:
Epoch 2/5
75110/75110 [==============================] - 20s - loss: 1.9348    
Epoch 3/5
75110/75110 [==============================] - 19s - loss: 1.8985    
Epoch 4/5
75110/75110 [==============================] - 20s - loss: 1.8667    
Epoch 5/5
75110/75110 [==============================] - 20s - loss: 1.8365    


### Test model

In [55]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [56]:
get_next('for thos')

'e'

In [57]:
get_next('part of ')

't'

In [58]:
get_next('queens a')

'n'

In [59]:
get_next('there is')

' '

### Our first RNN with Keras

In [60]:
n_hidden, n_fac, cs, vocab_size = (256, 42, 8, 86)

In [61]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=n_fac, input_length=cs),
    SimpleRNN(output_dim=n_hidden, activation='relu', inner_init='identity'), # what does identity mean?
    Dense(output_dim=vocab_size, activation='softmax')
])

In [62]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_4 (Embedding)          (None, 8, 42)         3612        embedding_input_1[0][0]          
____________________________________________________________________________________________________
simplernn_1 (SimpleRNN)          (None, 256)           76544       embedding_4[0][0]                
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 86)            22102       simplernn_1[0][0]                
Total params: 102258
____________________________________________________________________________________________________


In [63]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy')

In [64]:
model.fit(np.concatenate(xs, axis=1), y, batch_size=64, nb_epoch=4)

Epoch 1/4
75110/75110 [==============================] - 22s - loss: 2.8062    
Epoch 2/4
75110/75110 [==============================] - 20s - loss: 2.2971    
Epoch 3/4
75110/75110 [==============================] - 20s - loss: 2.0870    - 
Epoch 4/4
75110/75110 [==============================] - 21s - loss: 1.9542    


In [65]:
model.fit(np.concatenate(xs, axis=1), y, batch_size=64, nb_epoch=4)

Epoch 1/4
75110/75110 [==============================] - 19s - loss: 1.8530    
Epoch 2/4
75110/75110 [==============================] - 22s - loss: 1.7724    
Epoch 3/4
75110/75110 [==============================] - 21s - loss: 1.7012    
Epoch 4/4
75110/75110 [==============================] - 19s - loss: 1.6467    


In [66]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]